In [4]:
!pip install unstructured pypdf python-docx nltk --quiet  # Install Unstructured to extract structured content from PDFs

# Download NLTK tokenizer for sentence splitting (used later)
import nltk
nltk.download("punkt")# Required for sentence tokenization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 6.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Import and Load PDF

from unstructured.partition.pdf import partition_pdf
import os
from collections import Counter
import re


# Place path for PDF application
pdf_path = "your_application.pdf"

# Use Unstructured's partition_pdf to break the PDF into structured elements
elements = partition_pdf(filename=pdf_path)

# Preview total elements found
print(f"Total elements extracted: {len(elements)}")
print("First element preview:")
print(elements[0])

In [ ]:
# Step 1: Collect all short blocks from the document
# We're focusing on blocks with <10 words — likely to be headers, footers, or labels


from collections import Counter

short_blocks = [
    el.text.strip()
    for el in elements
    if hasattr(el, "text") and 0 < len(el.text.strip().split()) < 10
]

#  Count how often each short block appears
footer_counts = Counter(short_blocks)

# If something appears too often (e.g., >5 times), it's probably a footer/header
# You can tune this threshold depending on document size
common_footers = {
    text for text, count in footer_counts.items() if count > 5
}

# Optional: Inspect them
print(" Common footer/header candidates detected:\n")
for f in list(common_footers)[:10]:
    print(f"- {f}")


In [3]:

#  Define a helper function to detect bullet points
def is_bullet_point(text):
    """
    Returns True if a line looks like a bullet point.
    Matches:
      - • Conduct...
      - - Maintain...
      - * Submit...
      - 1. Evaluate...
      - 2) Review...
    """
    text = text.strip()
    return bool(re.match(r"^(\s*[\*\-•]\s+|\d+[\.\)]\s+)", text))


# Step 2: Filter out unwanted text
# Keep text blocks if they are either:
# - Long paragraphs (>15 words), OR
# - Bullet points (even short ones)

filtered_chunks = []
discarded_chunks = []

for el in elements:
    if hasattr(el, "text"):
        text = el.text.strip()
        word_count = len(text.split())

        if word_count > 15 or is_bullet_point(text):
            filtered_chunks.append(text)
        else:
            discarded_chunks.append(text)

# ---- Step 3: Show basic stats ----
print(f" Retained: {len(filtered_chunks)} meaningful paragraphs")
print(f" Discarded: {len(discarded_chunks)} short or noisy blocks")

# Optional: Preview a few filtered results
print("\n Sample Clean Paragraph:\n")
print(filtered_chunks[0][:500])

# Optional: Preview what was discarded (debugging)
print("\n Sample Discarded Text:\n")
print(discarded_chunks[:5])


Jello


In [ ]:
# === Save cleaned output to file (optional) ===
with open("clean_application.txt", "w", encoding="utf-8") as f:
    for para in filtered_chunks:
        f.write(para + "\n\n")

